### Import potrzebnych bibliotek

In [2]:
import pandas as pd

from bs4 import BeautifulSoup
import requests

import os
import time
import glob

### Pobieranie stron na dysk lokalny

Generator generuje (heh) adresy URL wg. wzorca {base_url}{i} tj. w tym przypadku *https://blog.prokulski.science/index.php/wp-json/nv/v1/posts/page/{1..}*
i zwraca treść odpowiedzi HTTP do momentu, aż odpowiedź będzie pusta, wystąpi błąd `RequestException` lub zostanie wygenerowana liczba zapytań równa wartości parametru `pages`


W odpowiedzi na żądanie HTTP zwracany jest JSON (nagłówek odpowiedzi ```Content-Type: application/json; charset=UTF-8```), należy więc użyć funkcji `json()`, aby otrzymać odpowiednio sformatowaną treść odpowiedzi.

In [15]:
def post_from_url_generator(base_url, pages=None, sleep_time=5):
    if pages is None:
        pages = 9999
    for i in range(1, pages+1):
        url = f"{base_url}{i}"
        try:
            response = requests.post(url)
        except requests.RequestException:
            break
        time.sleep(sleep_time)
        respone_json = response.json()
        if respone_json: yield respone_json
        else: break

Przykład użycia generatora - kolejne zapytanie wysyłane są dopiero przy kolejnym wywołaniu funkcji `next(gen)` bądź kiedy będziemy iterować po generatorze

In [21]:
base_url = "https://blog.prokulski.science/index.php/wp-json/nv/v1/posts/page/"
gen = post_from_url_generator(base_url)
page = next(gen)
print(page)

### Wyodrębnienie adresów do stron postów


Funkcja jako parametr przyjmuje treść dokumentu HTML i wyciąga wszystkie tagi `<a\>`, które znajdują się w tagu `<h2\>` a następnie wyciągamy atrybut `href`

In [ ]:
def get_post_urls_from_page(page: str) -> list:
    soup = BeautifulSoup(page, 'lxml')
    result_urls = []
    for x in soup.select("h2 > a"):
        url = x['href']
        result_urls.append(url)
    return result_urls

Przykład użycia

In [23]:
get_post_urls_from_page(page)

### Pobieranie postów na lokalną maszynę, aby podczas dalszej pracy nie obciążać serwera

Składamy listę z hiperłączy do postów uzyskanych z wcześniejszego generatora

In [ ]:
post_urls = list()
[post_urls.append(get_post_urls_from_page(x)) for x in post_from_url_generator(base_url)]
# post_urls = list(itertools.chain.from_iterable(post_urls))

Pobieramy wszystkie posty z wygenerowanych wcześniej linków. W tym przypadku nagłówek odpowiedzi ```Content-Type: text/html; charset=UTF-8``` nie wymaga używania funkcji `json()`, odpowiedź pobieramy z atrybutu `text`

Pobrane treści postów zapisujemy w folderze `pages/{data}_{tytuł}.html`


In [ ]:
for url in post_urls:
    post_page = requests.get(url).text
    time.sleep(2)
    post_title = url.replace("https://blog.prokulski.science/index.php/", "").replace("/", "_")[:-1]
    print(f'Saving post page in file: {post_title}.html')
    with open("pages/"+post_title+".html", "wb") as outfile:
        outfile.write(post_page)



### Parsowanie postów z BeautifulSoup

Funkcja ładuje do pamięci treści postów i tworzy na ich podstawie obiekty BeautifulSoup.

Jako parametr możemy podać parser, który zostanie wykorzystany przez BeautifulSoup. Dwa z popularniejszych parserów to ```html.parser```, który jest domyślnym parserem ```bs4``` oraz ```lxml``` (bodajże trzeba oddzielnie zainstalować), który powinien być znacznie szybszy niż domyślny ```html.parser```

In [30]:
def read_posts_from_disk(path: str, parser: str='html.parser'):
    files = glob.glob(path)
    posts = []

    for file in files:
        with open(file, "rb") as input_file:
            filename = os.path.basename(file)
            post_date = filename[:10].replace("_", "-")
            post_name = filename.replace(".html", "")
            soup = BeautifulSoup(input_file.read(), parser)
            posts.append((post_date, post_name, soup))
    return posts

Wywołujemy powyższą funkcję i wczytujemy posty (w przykładzie tylko posty z 2019 roku)

In [ ]:
posts = read_posts_from_disk("pages/2019*", parser='lxml')

Funkcje, którymi w następnym kroku wydobędziemy informacje o postach. Fukncje przyjmują jako argument obiekt BeautifulSoup

In [ ]:
def get_tags(soup: BeautifulSoup) -> list:
    # wybieramy wszystkie tagi <a>, które występują w tagu z klasą ".nv-tags-list"
    tag_links = soup.select(".nv-tags-list > a")
    # składamy wartości tekstowe tagów (tag.string) w listę
    return [x.string for x in tag_links]
    # return list(map(lambda x: x.string, tag_links))


def get_listings_count(soup: BeautifulSoup) -> int:
    # wybieramy wszystkie tagi, które mają klasę ".crayon-pre" oraz występują w tagu z klasą ".crayon-code"
    code_divs = soup.select(".crayon-code > .crayon-pre")
    # zwracamy liczbę znalezionych tagów
    return len(code_divs)


def get_code_lines_count(soup: BeautifulSoup) -> int:
    line_divs = soup.select(".crayon-code > .crayon-pre > .crayon-line")
    return len(line_divs)


def get_tables_count(soup: BeautifulSoup) -> int:
    # wybieramy wszystkie tagi <table>, które posiadają jakąkolwiek klasę z listy
    tables = soup.select("table:is(.table, .table-striped, .table-hover, .table-condensed, .table-responsive)")
    # zwracamy liczbę znalezionych tagów
    return len(tables)


def get_images_count(soup: BeautifulSoup):
    # wybieramy wszystkie tagi <img>, które występują w tagu o id "wtr-content"
    imgs = soup.select("#wtr-content > img")
    # zwracamy liczbę znalezionych tagów
    return len(imgs)


def get_comments(soup: BeautifulSoup):
    comments = []
    # wybieramy wszystkie tagi z klasą ".nv-comment-article"
    for comment in soup.select(".nv-comment-article"):
        comment_row = {}
        comment_row['author'] = comment.select(".comment-author .author")[0].get_text()
        comment_row['date'] = pd.to_datetime(comment.select("time.entry-date.published")[0]
                                     .text.replace(" o", ""))
        content_tags = comment.select("div.nv-comment-content.comment.nv-content-wrap > p")[0]
        comment_row['comment'] = content_tags.get_text()
        comments.append(comment_row)
    return comments


Z listy wczytanych postów wyciągamy informacje i tworzymy dwa obiekty ```DataFrame```, jeden dla postów a drugi dla komentarzy

In [ ]:
rows_list = []
comments_list = []
i=1
for date, title, soup in posts:
    row = {}
    # row['id'] = i
    row['title'] = title
    row['post_date'] = date
    row['code_lines'] = get_code_lines_count(soup)
    row['listings_num'] = get_listings_count(soup)
    row['tables_count'] = get_tables_count(soup)
    row['images_count'] = get_images_count(soup)
    row['tags'] = get_tags(soup)
    rows_list.append(row)

    comm = {'comments': get_comments(soup)}
    comments_list.append(comm)
    i += 1

post_df = pd.DataFrame(rows_list)
comment_df = pd.DataFrame(comments_list)

Nadajemy nazwy indeksom

In [ ]:
post_df.index.name = 'id'
comment_df.index.name = 'id'

Rozszerzamy ```DataFrame```, żeby jeden komentarz był w jednym wierszu

In [ ]:
exploded_comment_df = comment_df.explode('comments')

Tworzymy ```DataFrame``` z kolumny *comments*

In [ ]:
details_comment_df = exploded_comment_df['comments'].apply(pd.Series)

Usuwamy kolumnę `0` oraz wiersze bez komentarzy

In [ ]:
details_comment_df = details_comment_df.drop(0, axis=1)
details_comment_df = details_comment_df.dropna()

Wyświetlamy końcowy wynik

In [ ]:
post_df.head()

In [ ]:
details_comment_df.head()